# GASpy manuscript scratch work
This notebook contains the scratch work necessary to generate the various metrics and numbers that we used to create the manuscript for GASpy.

In [1]:
import dill as pickle
import numpy as np
import tqdm
from pymatgen.matproj.rest import MPRester
from tpot import TPOTRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn import metrics
from gaspy import gasdb, defaults, utils
from gaspy.utils import vasp_settings_to_str
from gaspy_regress.regressor import GASpyRegressor

# Enumeration of search space

We have some things in our catalog that we want to exclude from our analyses. We exclude Ca, Na, and Nb because they're pretty new to the catalog, so we have little data on them. We also exclude Se and S because we don't do spin polarization, so those results are probably (moderately) inaccurate.

In [2]:
excluded_elements = set(['Ca', 'Na', 'Nb', 'S', 'Se'])

In [4]:
# Fetch the entire catalog
docs = gasdb.get_docs(collection_name='catalog')

# Find all of the compositions of each MPID so that we can
# later parse out any sites whose bulks have elements on the exclusion list
mpids = set(doc['mpid'] for doc in docs)
composition_by_mpid = dict.fromkeys(mpids)
# Connect to MP
mp_key = utils.read_rc('matproj_api_key')
with MPRester(mp_key) as mp_db:
    for mpid in tqdm.tqdm(mpids):
        # Get the info from MP
        entry = mp_db.get_entry_by_material_id({'task_ids': mpid})
        # Store the composition
        comp = entry.as_dict()['composition'].keys()
        composition_by_mpid[mpid] = set(comp)
# Now filter the docs and mpids
filtered_docs = []
for doc in docs:
    mpid = doc['mpid']
    comp = composition_by_mpid[mpid]
    if not comp.intersection(excluded_elements):
        filtered_docs.append(doc)
docs = filtered_docs

# Find all the elements, mpids, the unique alloys, and the surfaces
mpids = set(doc['mpid'] for doc in docs)
elements = set(el for mpid in mpids for el in composition_by_mpid[mpid])
compounds = set(str(composition_by_mpid[mpid]) for mpid in mpids)
surfaces = set((doc['mpid'], str(doc['miller'])) for doc in docs)

print('We enumerated %i elements.' % len(elements))
print('We enumerated %i crystal structures.' % len(mpids))
print('We enumerated %i different componds.' % len(compounds))
print('We enumerated %i surfaces.' % len(surfaces))
print('We enumerated %i adsorption sites.' % len(docs))

Starting to pull documents...


4223308it [01:49, 38521.28it/s]
100%|██████████| 3399/3399 [06:58<00:00,  9.06it/s]


We enumerated 31 elements.
We enumerated 1499 crystal structures.
We enumerated 709 different componds.
We enumerated 17507 surfaces.
We enumerated 1684908 adsorption sites.


# How many results do we have?

In [4]:
# Find the total number of DFT simulations
docs = gasdb.get_docs()

# Find the number of simulations that passed our filters
filters = dict(energy_min=-4, energy_max=4, f_max=0.5,
               ads_move_max=1.5, bare_slab_move_max=0.5, slab_move_max=1.5)
filtered_docs = gasdb.get_docs(**filters)

# Find the number of calcs for CO and H
co_docs = gasdb.get_docs(adsorbates=['CO'], **filters)
h_docs = gasdb.get_docs(adsorbates=['H'], **filters)

print('We have completed %i DFT simulations, including all adsorbates' % len(docs))
print('%i of these simulations pass the exclusion criteria, including only CO and H'
      % (len(co_docs) + len(h_docs)))
print('%i of these simulations were on CO' % len(co_docs))
print('%i of these simulations were on H' % len(h_docs))

Starting to pull documents...


95813it [00:01, 74358.10it/s]


Starting to pull documents...


52317it [00:00, 69663.30it/s]


Starting to pull documents...


19644it [00:00, 65805.51it/s]


Starting to pull documents...


23141it [00:00, 64964.84it/s]

We have completed 95813 DFT simulations, including all adsorbates
42785 of these simulations pass the exclusion criteria, including only CO and H
19644 of these simulations were on CO
23141 of these simulations were on H


# Discovery of catalyst candidates

In [5]:
# Set the ranges we want to look at
ads = 'CO'
dE_low = -0.77
optimal = -0.67
dE_high = -0.57

# Pull the data from the databall, then filter out
# documents that contain excluded elements
with open('CO2RR_predictions.pkl', 'rb') as f:
    dft_results, ml_results = pickle.load(f)

# Find all of the compositions of each MPID so that we can
# later parse out any sites whose bulks have elements on the exclusion list
mpids = set(result[0]['mpid'] for result in dft_results+ml_results)
composition_by_mpid = dict.fromkeys(mpids)
mp_key = utils.read_rc('matproj_api_key')
with MPRester(mp_key) as mp_db:
    for mpid in tqdm.tqdm(mpids):
        entry = mp_db.get_entry_by_material_id({'task_ids': mpid})
        comp = entry.as_dict()['composition'].keys()
        composition_by_mpid[mpid] = set(comp)

# Define docs and parse out elements on the exclusion list
docs = []
for result in dft_results:
    doc = result[0]
    mpid = doc['mpid']
    comp = composition_by_mpid[mpid]
    if not comp.intersection(excluded_elements):
        docs.append(doc)

# Find and report the number of near-optimal surfaces and componds
optimal_surfaces = [doc for doc in docs
                    if dE_low < doc['energy'] < dE_high]
optimal_compounds = set(str(composition_by_mpid[doc['mpid']])
                        for doc in optimal_surfaces)
print('We have %i surfaces with %s binding energies between %.2f and %.2f eV,'
      % (len(optimal_surfaces), ads, dE_low, dE_high))
print('which corresponds to %i different compounds' % len(optimal_compounds))

100%|██████████| 3403/3403 [06:25<00:00,  6.49it/s]


We have 130 surfaces with CO binding energies between -0.77 and -0.57 eV,
which corresponds to 54 different compounds


In [6]:
# Set the ranges we want to look at
ads = 'H'
dE_low = -0.37
optimal = -0.27
dE_high = -0.17

# Pull the data from the databall, then filter out
# documents that contain excluded elements
with open('HER_predictions.pkl', 'rb') as f:
    dft_results, ml_results = pickle.load(f)

# Find all of the compositions of each MPID so that we can
# later parse out any sites whose bulks have elements on the exclusion list
mpids = set(result[0]['mpid'] for result in dft_results+ml_results)
composition_by_mpid = dict.fromkeys(mpids)
mp_key = utils.read_rc('matproj_api_key')
with MPRester(mp_key) as mp_db:
    for mpid in tqdm.tqdm(mpids):
        entry = mp_db.get_entry_by_material_id({'task_ids': mpid})
        comp = entry.as_dict()['composition'].keys()
        composition_by_mpid[mpid] = set(comp)

# Define docs and parse out elements on the exclusion list
docs = []
for result in dft_results:
    doc = result[0]
    mpid = doc['mpid']
    comp = composition_by_mpid[mpid]
    if not comp.intersection(excluded_elements):
        docs.append(doc)

# Find and report the number of near-optimal surfaces and componds
optimal_surfaces = [doc for doc in docs
                    if dE_low < doc['energy'] < dE_high]
optimal_compounds = set(str(composition_by_mpid[doc['mpid']])
                        for doc in optimal_surfaces)
print('We have %i surfaces with %s binding energies between %.2f and %.2f eV,'
      % (len(optimal_surfaces), ads, dE_low, dE_high))
print('which corresponds to %i different compounds' % len(optimal_compounds))

100%|██████████| 3404/3404 [06:11<00:00,  8.95it/s]

We have 258 surfaces with H binding energies between -0.37 and -0.17 eV,
which corresponds to 102 different compounds
